<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Preprocess-Data" data-toc-modified-id="Preprocess-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocess Data</a></span><ul class="toc-item"><li><span><a href="#Convert-Score-Into-Grade" data-toc-modified-id="Convert-Score-Into-Grade-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Convert Score Into Grade</a></span></li><li><span><a href="#Map-Grade-To-Number" data-toc-modified-id="Map-Grade-To-Number-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Map Grade To Number</a></span></li><li><span><a href="#Deal-With-The-NaN-Value" data-toc-modified-id="Deal-With-The-NaN-Value-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Deal With The NaN Value</a></span></li><li><span><a href="#Deal-With-Non-Numeric-Column" data-toc-modified-id="Deal-With-Non-Numeric-Column-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Deal With Non Numeric Column</a></span></li><li><span><a href="#Drop-Unnecessary-Column" data-toc-modified-id="Drop-Unnecessary-Column-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Drop Unnecessary Column</a></span></li><li><span><a href="#Split-Training-Set-And-Test-Set" data-toc-modified-id="Split-Training-Set-And-Test-Set-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Split Training Set And Test Set</a></span></li></ul></li><li><span><a href="#Create-Dataset-Class" data-toc-modified-id="Create-Dataset-Class-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create Dataset Class</a></span><ul class="toc-item"><li><span><a href="#Create-Training-Set-Class" data-toc-modified-id="Create-Training-Set-Class-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Create Training Set Class</a></span></li><li><span><a href="#Create-Test-Set-Class" data-toc-modified-id="Create-Test-Set-Class-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Create Test Set Class</a></span></li></ul></li><li><span><a href="#Create-Dataloader" data-toc-modified-id="Create-Dataloader-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create Dataloader</a></span></li><li><span><a href="#Define-Model" data-toc-modified-id="Define-Model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Define Model</a></span></li><li><span><a href="#Test-Performance" data-toc-modified-id="Test-Performance-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Test Performance</a></span></li></ul></div>

##### Load Package

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import random

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

##### Set Up Seed

In [4]:
def setup_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministric = True

In [5]:
setup_seed(9)

# Load Data

In [6]:
df = pd.read_excel("Financial_E.xlsx")

In [7]:
df.head()

,Organization ID,Fiscal Year,Ticker,Feed Date,Fiscal Year End Date,CO2 Equivalents Emission Total,CO2 Equivalents Emission Direct,CO2 Equivalents Emission Indirect,Flaring of Natural Gas,Cement CO2 Equivalents Emission,...,Nuclear,Environmental Resource Impact Controversies,Recent Environmental Resource Impact Controversies,Environmental Supply Chain Monitoring,Real Estate Sustainability Certification,ISO 14000 or EMS,CO2 estimation method,TRBC used for Median Calculation,Unnamed: 193,Environmental Score
0,18183,2002,AXP,2020-08-02,2002-12-31,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,0.180989
1,18183,2003,AXP,2020-08-02,2003-12-31,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,0.161219
2,18183,2004,AXP,2020-08-02,2004-12-31,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,0.180355
3,18183,2005,AXP,2020-08-02,2005-12-31,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,0.149305
4,18183,2006,AXP,2020-08-02,2006-12-31,220996.55,9082.05,211914.5,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,0.613290


In [8]:
df.tail()

,Organization ID,Fiscal Year,Ticker,Feed Date,Fiscal Year End Date,CO2 Equivalents Emission Total,CO2 Equivalents Emission Direct,CO2 Equivalents Emission Indirect,Flaring of Natural Gas,Cement CO2 Equivalents Emission,...,Nuclear,Environmental Resource Impact Controversies,Recent Environmental Resource Impact Controversies,Environmental Supply Chain Monitoring,Real Estate Sustainability Certification,ISO 14000 or EMS,CO2 estimation method,TRBC used for Median Calculation,Unnamed: 193,Environmental Score
696,114217052,2016,SYF,2021-05-30,2016-12-31,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,Median,55101030.0,NaN,0.146550
697,114217052,2017,SYF,2022-04-10,2017-12-31,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,Median,55101030.0,NaN,0.156625
698,114217052,2018,SYF,2021-07-11,2018-12-31,27487.0,1751.0,25736.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,Reported,NaN,NaN,0.195295
699,114217052,2019,SYF,2022-04-10,2019-12-31,26799.0,1974.0,24825.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,Reported,NaN,NaN,0.139922
700,114217052,2020,SYF,2022-04-24,2020-12-31,17385.0,1167.0,16218.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,Reported,NaN,NaN,0.623711


In [9]:
df.shape

(701, 195)

In [10]:
df.dtypes

Organization ID                              int64
Fiscal Year                                  int64
Ticker                                      object
Feed Date                           datetime64[ns]
Fiscal Year End Date                datetime64[ns]
                                         ...      
ISO 14000 or EMS                            object
CO2 estimation method                       object
TRBC used for Median Calculation           float64
Unnamed: 193                               float64
Environmental Score                        float64
Length: 195, dtype: object

# Preprocess Data

## Convert Score Into Grade

In [11]:
def convert_score_t_grade(score):
    if 0 <= score <= 0.083333:
        return "D-"
    
    elif 0.083333 < score <= 0.166666:
        return "D"
    
    elif 0.166666 < score <= 0.250000:
        return "D+"
    
    elif 0.250000 < score <= 0.333333:
        return "C-"
    
    elif 0.333333 < score <= 0.416666:
        return "C"
    
    elif 0.416666 < score <= 0.500000:
        return "C+"
    
    elif 0.500000 < score <= 0.583333:
        return "B-"
    
    elif 0.583333 < score <= 0.666666:
        return "B+"
    
    elif 0.666666 < score <= 0.750000:
        return "B+"
    
    elif 0.750000 < score <= 0.833333:
        return "A-"
    
    elif 0.833333 < score <= 0.916666:
        return "A"
    
    elif 0.916666 < score <= 1:
        return "A+"

In [12]:
df["E_Grade"] = df["Environmental Score"].apply(convert_score_t_grade)

In [13]:
df["E_Grade"].value_counts()

D     188
A+    133
A      89
D+     88
B+     75
A-     52
C      24
B-     23
C-     16
C+     12
Name: E_Grade, dtype: int64

## Map Grade To Number

In [14]:
grade_to_number = {"A+": 0,
                   "A": 1,
                   "A-": 2,
                   "B+": 3,
                   "B": 4,
                   "B-": 5,
                   "C+": 6,
                   "C": 7,
                   "C-": 8,
                   "D+": 9,
                   "D": 10, 
                   "D-": 11}

In [15]:
df["E_num"] = df["E_Grade"].map(grade_to_number)

In [16]:
df

,Organization ID,Fiscal Year,Ticker,Feed Date,Fiscal Year End Date,CO2 Equivalents Emission Total,CO2 Equivalents Emission Direct,CO2 Equivalents Emission Indirect,Flaring of Natural Gas,Cement CO2 Equivalents Emission,...,Recent Environmental Resource Impact Controversies,Environmental Supply Chain Monitoring,Real Estate Sustainability Certification,ISO 14000 or EMS,CO2 estimation method,TRBC used for Median Calculation,Unnamed: 193,Environmental Score,E_Grade,E_num
0,18183,2002,AXP,2020-08-02,2002-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,0.180989,D+,9.0
1,18183,2003,AXP,2020-08-02,2003-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,0.161219,D,10.0
2,18183,2004,AXP,2020-08-02,2004-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,0.180355,D+,9.0
3,18183,2005,AXP,2020-08-02,2005-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,0.149305,D,10.0
4,18183,2006,AXP,2020-08-02,2006-12-31,220996.55,9082.05,211914.5,NaN,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,0.613290,B+,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,114217052,2016,SYF,2021-05-30,2016-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,No,Median,55101030.0,NaN,0.146550,D,10.0
697,114217052,2017,SYF,2022-04-10,2017-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,No,Median,55101030.0,NaN,0.156625,D,10.0
698,114217052,2018,SYF,2021-07-11,2018-12-31,27487.00,1751.00,25736.0,NaN,NaN,...,NaN,NaN,NaN,No,Reported,NaN,NaN,0.195295,D+,9.0
699,114217052,2019,SYF,2022-04-10,2019-12-31,26799.00,1974.00,24825.0,NaN,NaN,...,NaN,NaN,NaN,No,Reported,NaN,NaN,0.139922,D,10.0


## Deal With The NaN Value

In [17]:
df = df.fillna(0)

In [18]:
df

,Organization ID,Fiscal Year,Ticker,Feed Date,Fiscal Year End Date,CO2 Equivalents Emission Total,CO2 Equivalents Emission Direct,CO2 Equivalents Emission Indirect,Flaring of Natural Gas,Cement CO2 Equivalents Emission,...,Recent Environmental Resource Impact Controversies,Environmental Supply Chain Monitoring,Real Estate Sustainability Certification,ISO 14000 or EMS,CO2 estimation method,TRBC used for Median Calculation,Unnamed: 193,Environmental Score,E_Grade,E_num
0,18183,2002,AXP,2020-08-02,2002-12-31,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,No,0,0.0,0.0,0.180989,D+,9.0
1,18183,2003,AXP,2020-08-02,2003-12-31,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,No,0,0.0,0.0,0.161219,D,10.0
2,18183,2004,AXP,2020-08-02,2004-12-31,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,No,0,0.0,0.0,0.180355,D+,9.0
3,18183,2005,AXP,2020-08-02,2005-12-31,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,No,0,0.0,0.0,0.149305,D,10.0
4,18183,2006,AXP,2020-08-02,2006-12-31,220996.55,9082.05,211914.5,0.0,0.0,...,0.0,0.0,0.0,No,0,0.0,0.0,0.613290,B+,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,114217052,2016,SYF,2021-05-30,2016-12-31,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,No,Median,55101030.0,0.0,0.146550,D,10.0
697,114217052,2017,SYF,2022-04-10,2017-12-31,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,No,Median,55101030.0,0.0,0.156625,D,10.0
698,114217052,2018,SYF,2021-07-11,2018-12-31,27487.00,1751.00,25736.0,0.0,0.0,...,0.0,0.0,0.0,No,Reported,0.0,0.0,0.195295,D+,9.0
699,114217052,2019,SYF,2022-04-10,2019-12-31,26799.00,1974.00,24825.0,0.0,0.0,...,0.0,0.0,0.0,No,Reported,0.0,0.0,0.139922,D,10.0


## Deal With Non Numeric Column

In [19]:
df = df.set_index(["Fiscal Year", "Ticker"])

In [20]:
df = df.select_dtypes(["int64", "float64"])

## Drop Unnecessary Column

In [21]:
df = df.drop(columns = ["Organization ID", "Environmental Score"])

In [22]:
df

,,CO2 Equivalents Emission Total,CO2 Equivalents Emission Direct,CO2 Equivalents Emission Indirect,Flaring of Natural Gas,Cement CO2 Equivalents Emission,Ozone-Depleting Substances,NOx Emissions,SOx Emissions,VOC Emissions,Waste Total,...,Land Environmental Impact Reduction,Resource Reduction Commitment (Inactive),Nuclear,Environmental Resource Impact Controversies,Recent Environmental Resource Impact Controversies,Environmental Supply Chain Monitoring,Real Estate Sustainability Certification,TRBC used for Median Calculation,Unnamed: 193,E_num
Fiscal Year,Ticker,,,,,,,,,,,,,,,,,,,,,
2002,AXP,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
2003,AXP,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
2004,AXP,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
2005,AXP,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
2006,AXP,220996.55,9082.05,211914.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,SYF,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,55101030.0,0.0,10.0
2017,SYF,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,55101030.0,0.0,10.0
2018,SYF,27487.00,1751.00,25736.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,9.0


In [23]:
count_zero = (df["CO2 Equivalents Emission Indirect"] == 0).sum()

count_zero

377

In [24]:
df = df.loc[:, (df==0).mean() <0.7]

## Split Training Set And Test Set

In [25]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, :],
                                                    df.iloc[:, -1],
                                                    test_size = 0.1,
                                                    random_state = 42)

print(x_train.shape)
print(x_test.shape)

(630, 30)
(71, 30)


# Create Dataset Class

## Create Training Set Class

In [26]:
class TrainingSet(Dataset):
    
    def __init__(self):
        xy = x_train.values
        
        self.x = xy[:, :-1].astype(np.float32)
        #self.x = scaler.fit_transform(self.x)
        self.y = xy[:, -1].astype(np.int64)
        
        self.x = torch.from_numpy(self.x)
        self.y = torch.from_numpy(self.y)
        
        self.n_samples = xy.shape[0]
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples

In [27]:
training_set = TrainingSet()

In [28]:
first_data_training = training_set[0]

feature, label = first_data_training
print(feature.shape)
print(label)

torch.Size([29])
tensor(10)


## Create Test Set Class

In [29]:
class TestSet(Dataset):
    
    def __init__(self):
        xy = x_test.values
        
        self.x = xy[:, : -1].astype(np.float32)
        #self.x = scaler.fit_transform(self.x)
        self.y = xy[:, -1].astype(np.int64)
        
        self.x = torch.from_numpy(self.x)
        self.y = torch.from_numpy(self.y)
        
        self.n_samples = xy.shape[0]
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples

In [30]:
test_set = TestSet()

In [31]:
first_data_test = test_set[0]

feature, label = first_data_test
print(feature.shape)
print(label)

torch.Size([29])
tensor(3)


# Create Dataloader

In [32]:
train_loader = DataLoader(dataset = training_set,
                          batch_size = 40,
                          shuffle = True)

test_loader = DataLoader(dataset = test_set,
                         batch_size = 40,
                         shuffle = True)

In [33]:
dataiter = iter(train_loader)

data = dataiter.next()

In [34]:
features, labels = data

print(features.shape)
print(labels.shape)

torch.Size([40, 29])
torch.Size([40])


# Define Model

In [35]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        
        self.lin_start = nn.Linear(input_size, 800)
        #self.lin1 = nn.Linear(800, 400)
        self.lin_end = nn.Linear(800, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.lin_start(x)
        out = self.relu(out)
        
        #out = self.lin1(out)
        #out = self.relu(out)
        
        out = self.lin_end(out)
        
        return out

In [36]:
input_size = 29
output_size = 12

model = NeuralNetwork(input_size, output_size)

In [37]:
criterion = nn.CrossEntropyLoss()

In [38]:
learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate)

In [39]:
num_epochs = 300

for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):
        
        y_pred = model(features)
        loss = criterion(y_pred, labels)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if (i+1)%5 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step{i+1}/{len(train_loader)}, loss = {loss.item():.4f}')

epoch 1/300, step5/16, loss = 727548.9375
epoch 1/300, step10/16, loss = 27730.0215
epoch 1/300, step15/16, loss = 91704.3281
epoch 2/300, step5/16, loss = 36786.5352
epoch 2/300, step10/16, loss = 52347.8672
epoch 2/300, step15/16, loss = 26245.1777
epoch 3/300, step5/16, loss = 8004.4873
epoch 3/300, step10/16, loss = 25201.8711
epoch 3/300, step15/16, loss = 27322.5039
epoch 4/300, step5/16, loss = 9832.5391
epoch 4/300, step10/16, loss = 7934.4219
epoch 4/300, step15/16, loss = 15560.7188
epoch 5/300, step5/16, loss = 15928.0078
epoch 5/300, step10/16, loss = 12223.4521
epoch 5/300, step15/16, loss = 28045.3594
epoch 6/300, step5/16, loss = 27562.4258
epoch 6/300, step10/16, loss = 11221.5303
epoch 6/300, step15/16, loss = 6418.9404
epoch 7/300, step5/16, loss = 24669.9453
epoch 7/300, step10/16, loss = 18434.4922
epoch 7/300, step15/16, loss = 37025.0859
epoch 8/300, step5/16, loss = 29209.0469
epoch 8/300, step10/16, loss = 4097.6206
epoch 8/300, step15/16, loss = 3993.3530
epoch

epoch 68/300, step10/16, loss = 1548.8524
epoch 68/300, step15/16, loss = 4235.7197
epoch 69/300, step5/16, loss = 10797.4150
epoch 69/300, step10/16, loss = 1154.2925
epoch 69/300, step15/16, loss = 3640.7905
epoch 70/300, step5/16, loss = 2982.2905
epoch 70/300, step10/16, loss = 7789.6069
epoch 70/300, step15/16, loss = 1833.8129
epoch 71/300, step5/16, loss = 1949.8477
epoch 71/300, step10/16, loss = 1874.6332
epoch 71/300, step15/16, loss = 6662.3682
epoch 72/300, step5/16, loss = 4707.9941
epoch 72/300, step10/16, loss = 4212.1548
epoch 72/300, step15/16, loss = 4941.6816
epoch 73/300, step5/16, loss = 1613.4790
epoch 73/300, step10/16, loss = 2125.3696
epoch 73/300, step15/16, loss = 11146.4766
epoch 74/300, step5/16, loss = 12185.6055
epoch 74/300, step10/16, loss = 14495.1484
epoch 74/300, step15/16, loss = 2554.2041
epoch 75/300, step5/16, loss = 3652.6882
epoch 75/300, step10/16, loss = 2266.6367
epoch 75/300, step15/16, loss = 2805.2578
epoch 76/300, step5/16, loss = 3640.0

epoch 135/300, step5/16, loss = 9115.3477
epoch 135/300, step10/16, loss = 8330.4316
epoch 135/300, step15/16, loss = 5567.0835
epoch 136/300, step5/16, loss = 1452.2505
epoch 136/300, step10/16, loss = 9031.3525
epoch 136/300, step15/16, loss = 4599.8975
epoch 137/300, step5/16, loss = 557.1349
epoch 137/300, step10/16, loss = 5081.3350
epoch 137/300, step15/16, loss = 2940.4463
epoch 138/300, step5/16, loss = 800.1623
epoch 138/300, step10/16, loss = 2839.5540
epoch 138/300, step15/16, loss = 1979.0344
epoch 139/300, step5/16, loss = 2642.5537
epoch 139/300, step10/16, loss = 2460.9731
epoch 139/300, step15/16, loss = 2040.5338
epoch 140/300, step5/16, loss = 2332.3132
epoch 140/300, step10/16, loss = 315.9631
epoch 140/300, step15/16, loss = 8620.8379
epoch 141/300, step5/16, loss = 11219.2520
epoch 141/300, step10/16, loss = 4394.6265
epoch 141/300, step15/16, loss = 1454.7942
epoch 142/300, step5/16, loss = 3299.0364
epoch 142/300, step10/16, loss = 11039.4229
epoch 142/300, step1

epoch 199/300, step10/16, loss = 2411.7898
epoch 199/300, step15/16, loss = 2544.6309
epoch 200/300, step5/16, loss = 794.3292
epoch 200/300, step10/16, loss = 7775.1851
epoch 200/300, step15/16, loss = 3911.3218
epoch 201/300, step5/16, loss = 1574.2565
epoch 201/300, step10/16, loss = 15346.9785
epoch 201/300, step15/16, loss = 1604.8385
epoch 202/300, step5/16, loss = 3274.8586
epoch 202/300, step10/16, loss = 3420.6516
epoch 202/300, step15/16, loss = 2083.6685
epoch 203/300, step5/16, loss = 10601.8574
epoch 203/300, step10/16, loss = 3485.5442
epoch 203/300, step15/16, loss = 2993.7603
epoch 204/300, step5/16, loss = 6582.8096
epoch 204/300, step10/16, loss = 6273.1099
epoch 204/300, step15/16, loss = 3299.9082
epoch 205/300, step5/16, loss = 9545.9395
epoch 205/300, step10/16, loss = 3367.0991
epoch 205/300, step15/16, loss = 705.0094
epoch 206/300, step5/16, loss = 3119.5020
epoch 206/300, step10/16, loss = 6058.9707
epoch 206/300, step15/16, loss = 8700.5010
epoch 207/300, ste

epoch 267/300, step5/16, loss = 401.5513
epoch 267/300, step10/16, loss = 819.0840
epoch 267/300, step15/16, loss = 571.2174
epoch 268/300, step5/16, loss = 1073.2251
epoch 268/300, step10/16, loss = 1564.0594
epoch 268/300, step15/16, loss = 2672.2485
epoch 269/300, step5/16, loss = 737.8744
epoch 269/300, step10/16, loss = 2147.2688
epoch 269/300, step15/16, loss = 1194.5363
epoch 270/300, step5/16, loss = 2104.8911
epoch 270/300, step10/16, loss = 3846.0320
epoch 270/300, step15/16, loss = 3238.5105
epoch 271/300, step5/16, loss = 1529.4810
epoch 271/300, step10/16, loss = 3535.2708
epoch 271/300, step15/16, loss = 1665.8994
epoch 272/300, step5/16, loss = 531.4811
epoch 272/300, step10/16, loss = 1238.8942
epoch 272/300, step15/16, loss = 1057.7026
epoch 273/300, step5/16, loss = 970.9141
epoch 273/300, step10/16, loss = 2103.5100
epoch 273/300, step15/16, loss = 891.3759
epoch 274/300, step5/16, loss = 6226.4277
epoch 274/300, step10/16, loss = 5512.6406
epoch 274/300, step15/16, 

# Test Performance

In [40]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    
    for features_t, labels_t in test_loader:
        
        y_pred_t = model(features_t)
        
        _, predictions = torch.max(y_pred_t, 1)
        n_samples += labels_t.shape[0]
        n_correct += (predictions == labels_t).sum().item()
    
    acc = 100*n_correct/n_samples
    
    print(f'accuracy = {acc}')

accuracy = 46.478873239436616
